In [1]:
import pandas as pd
import numpy as np
import re

#Заранее зададим форматирование для колонки ur_inn_kpp
pd.options.display.float_format = '{:.0f}'.format

In [2]:
# Загрузим csv-файл с данными для заданий 1 и 2
dashboard_actions_df = pd.read_csv('dashbrd_action.csv', delimiter=';', decimal=',', index_col=False)

# 1) Для каждого топ 20 заказчиков по частоте размещений выбрать топ 3 поставщиков по числу побед

   ### использовать поле actiontype_nm
   ### считать уникальной компанию по полям ur_inn_kpp

In [3]:
# Копируем исходный датафрейм. Преобразуем время в подходящий формат
actions_df = dashboard_actions_df.copy()

In [4]:
# Отберем топ-20 заказчиков по частоте размещений, 
# для этого в поле actiontype_nm оставим только заказчиков и сгруппируем по полю "ur_inn_kpp"
actions_client_df =  actions_df[actions_df['actiontype_nm'] == 'заказчик']

top20_clients = actions_client_df.groupby('ur_inn_kpp', as_index=False).agg({'actiontype_nm' : 'count'})

#Отсортируем по убыванию, оставим топ-20, переименуем колонки
top20_clients = top20_clients.sort_values('actiontype_nm', ascending=False).head(20).reset_index(drop=True)
top20_clients.columns = ['ИНН', 'Размещений']

In [5]:
# Отобразим топ-5 клиентов для понимания того, как выглядит массив данных
top20_clients.head()

,ИНН,Размещений
0,7810810000000000000,20415
1,6663010000000000000,18268
2,7817300000000000000,14182
3,5001000000000000000,10316
4,5501070000000000000,9902


In [6]:
# Оставляем записи, в которых заказчиком являлся кто-то из топ-20, переименовываем колонку ИНН для последующего слияния
actions_by_top20_clients_df = actions_client_df[(actions_client_df['ur_inn_kpp'].isin(top20_clients['ИНН']))]

actions_by_top20_clients_df = actions_by_top20_clients_df.\
                                rename(columns= {'ur_inn_kpp' : 'ИНН Заказчика'})[['lot_id_bas','ИНН Заказчика']]

In [7]:
# Создадим массив записей о победителях
actions_winner_df = actions_df[actions_df['actiontype_nm'] == 'победитель']

In [8]:
# Соединяем победителей и топ-20 клиентов по лотам, группируем по паре ИНН заказчик-победитель
actions_customers_with_winners = actions_winner_df.merge(actions_by_top20_clients_df, on='lot_id_bas').\
groupby(by=['ИНН Заказчика','ur_inn_kpp'],as_index=False).\
agg({'actiontype_nm' : 'count'}).\
sort_values(by=['ИНН Заказчика','actiontype_nm'],ascending=False).\
rename(columns={"ur_inn_kpp": "ИНН Победителя", "actiontype_nm": "Число побед Победителя"})

In [9]:
# Оставляем только топ-3 вендора для каждого из 20 заказчиков
top_clients_with_vendors = actions_customers_with_winners.set_index('ИНН Победителя').\
groupby("ИНН Заказчика")['Число побед Победителя'].\
nlargest(3).\
reset_index()

In [10]:
# Вернемся к очередности по общему количеству размещений заказчика
top_clients_with_vendors_ordered = top20_clients.merge(top_clients_with_vendors,
                                               left_on='ИНН', right_on='ИНН Заказчика')[top_clients_with_vendors.columns]

In [11]:
# Смотрим на первые 6 строк полученного массива
top_clients_with_vendors_ordered.head(6)

,ИНН Заказчика,ИНН Победителя,Число побед Победителя
0,7810810000000000000,7719630000000000000,511
1,7810810000000000000,5047130000000000000,209
2,7810810000000000000,7743870000000000000,206
3,6663010000000000000,6673090000000000000,201
4,6663010000000000000,7817040000000000000,177
5,6663010000000000000,6673100000000000000,163


In [12]:
# Видим, что для ИНН 5501070000000000000 нет записей в конечном массиве данных
set(top20_clients['ИНН'].tolist()) - set(top_clients_with_vendors['ИНН Заказчика'].unique().tolist())

{5.50107e+18}

In [13]:
# Отбираем все события по лотам, где заказчиком был ИНН 5501070000000000000
lost_inn_actions = actions_df[(actions_df['ur_inn_kpp']==5501070000000000000) &
                  (actions_df['actiontype_nm']=='заказчик')]

In [14]:
# Видим, что в 9902 его размещениях ни разу не было победителя
actions_df[actions_df['lot_id_bas'].isin(lost_inn_actions['lot_id_bas'].unique().tolist())]['actiontype_nm'].value_counts()

заказчик       9902
организатор    9816
Name: actiontype_nm, dtype: int64

# 2) Топ 5 заказчиков по количеству на каждый месяц за прошедшие пол года
### мерджить файлы по полям lot_id_bas, section_nm 
### Дата смотреть по полю timestamp
### количество - по количеству lot_id_bas+actiontype_nm+ur_inn_kpp


In [15]:
# Копируем исходный массив данных
actions_df_2 = dashboard_actions_df.copy()

# Преобразуем время в подходящий формат
actions_df_2['timestamp'] = pd.to_datetime(actions_df_2['timestamp'].str[:-6], format='%Y-%m-%d %H:%M:%S')

In [16]:
# Смотрим максимальную дату. За прошедшие полгода информации нет, будем рассматривать последние доступные полгода
actions_df_2['timestamp'].max()

Timestamp('2020-12-24 00:00:00')

In [17]:
# Оставляем только записи 7 и более месяца 2020 года с действующим лицом "заказчик"
actions_df_2 = actions_df_2[(actions_df_2['timestamp'].dt.year==2020) &
                            (actions_df_2['timestamp'].dt.month>=7)   &
                            (actions_df_2['actiontype_nm'] == 'заказчик')]

In [18]:
# Вычленяем месяц из даты
actions_df_2['Месяц'] = actions_df_2['timestamp'].dt.month

In [19]:
# Группируем по месяцу, ИНН
actions_df_2_grouped = actions_df_2.groupby(by=['Месяц','ur_inn_kpp'],as_index=False).\
                        agg({'actiontype_nm' : 'count'}).sort_values(by=['Месяц','actiontype_nm'],ascending=False).\
                        rename(columns={'ur_inn_kpp' : 'ИНН', 'actiontype_nm':'Число размещений'})

In [20]:
# Оставляем только топ-5 заказчиков по числу размещений по месяцам
actions_df_2_grouped = actions_df_2_grouped.set_index('ИНН').groupby("Месяц")['Число размещений'].nlargest(5).reset_index()

In [21]:
# Смотрим первые 10 строк для понимания результата
actions_df_2_grouped.head(10)

,Месяц,ИНН,Число размещений
0,7,5501070000000000000,893
1,7,5260140000000000000,453
2,7,7810810000000000000,361
3,7,6663010000000000000,239
4,7,6673090000000000000,225
5,8,5260140000000000000,628
6,8,7810810000000000000,398
7,8,5501070000000000000,324
8,8,6665000000000000000,240
9,8,266049000000000000,200


# 3) Выделить из комментариев номер процедуры

In [22]:
# Считываем файл
procedures_df = pd.read_excel('Задание_аналитик.xlsx')

In [23]:
# Найдём все числа в каждой записи, выберем наибольшее из них (так как часто встречается однозначный номер лота)
numbers_df = procedures_df.copy()

numbers_df['Номер процедуры'] = numbers_df['Содержание'].apply(
                                lambda x: sorted(
                                    [int(y) for y in re.findall(r'\d+', x)],
                                    reverse=True)).\
                                str[0]

In [24]:
# Проверим количество записей, где, скорее всего, фигурирует номер лота или тарифа в отсутствие номера процедуры. 
numbers_df[numbers_df['Номер процедуры']<=1000]['Номер процедуры'].value_counts()

5      27
4      27
9      25
360    24
1      11
10      7
8       6
6       6
90      2
2       2
7       1
365     1
3       1
Name: Номер процедуры, dtype: int64

In [25]:
# Заменяем номера лотов, тарифов и записи без чисел на "Отсутствует"
numbers_df.iloc[numbers_df[numbers_df['Номер процедуры']<=1000].index.tolist(),
                numbers_df.columns.get_loc('Номер процедуры')] = 'Отсутствует'

numbers_df['Номер процедуры'] = numbers_df['Номер процедуры'].fillna('Отсутствует')

In [26]:
# Смотрим первые 5 строк для понимания результата
numbers_df.head()

,Содержание,Номер процедуры
0,За участие в электронной процедуре (Закупка ма...,152213
1,Участие в электронной процедуре (Конкурс в эле...,32110882035
2,За участие в электронной процедуре (Закупка ма...,151059
3,Участие в электронной процедуре (Запрос предло...,32110998113
4,За участие в электронной процедуре (Закупка ма...,139657
